In [1]:
print("Success!")

Success!


# Load Dataset and Model

In [29]:
from datasets import load_dataset

file_path = "../data/to_annotate_150_cleaned.json"
dataset = load_dataset("json", data_files=file_path, split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/9130/.cache/huggingface/datasets/json/default-c0867ada11e1561a/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

# Version 2-7b
# base_model_name = "meta-llama/Llama-2-7b-hf"
# Version 2-13b-chat
base_model_name = "meta-llama/Llama-2-13b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

device_map = {"": 0}

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True,
    use_auth_token=True
)

base_model.config.use_cache = False

# More info: https://github.com/huggingface/transformers/pull/24906
base_model.config.pretraining_tp = 1 

/usr/local/lib/python3.10/site-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# Few-shot Inference

In [14]:
# System Prompt
system_prompt = "You are a helpful, respectful and honest assistant in sentence polarization analysis. \
And you are supposed to classify the sentiment polarity of the user's message about gun supoort into one of the following categories: \
'Support Gun Polarized', 'Support Gun non-Polarized', 'Neutral', 'Anti Gun Polarized', 'Anti Gun non-Polarized', 'Not relevant' or 'Not Sure'."
# User Prompt
user_prompt = "Please classify the sentiment polarity of the following sentence about gun supoort into one of the following categories: \
'Support Gun Polarized', 'Support Gun non-Polarized', 'Neutral', 'Anti Gun Polarized', 'Anti Gun non-Polarized', 'Not relevant' or 'Not Sure':"
# Examples
examples = """
Here is an inference example:
Sentence: we must ban this horrible weapon of war
Sentiment Polarity: Anti Gun Polarized

Here is another inference example:
Sentence: an armed society is a polite society , keep it up friends.
Sentiment Polarity: Support Gun non-Polarized
"""

In [18]:
sentence = "not every act of violence needs to be about gun law , i see two idiots here ."

In [20]:
text = user_prompt + "\n" + examples + "\n" + "Sentence: " + sentence + "\n" + "Sentiment Polarity:"

inputs = tokenizer(text, return_tensors="pt").to("cuda")
outputs = base_model.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=150, pad_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Please classify the sentiment polarity of the following sentence about gun supoort into one of the following categories: 'Support Gun Polarized', 'Support Gun non-Polarized', 'Neutral', 'Anti Gun Polarized', 'Anti Gun non-Polarized', 'Not relevant' or 'Not Sure':

Here is an inference example:
Sentence: we must ban this horrible weapon of war
Sentiment Polarity: Anti Gun Polarized

Here is another inference example:
Sentence: an armed society is a polite society , keep it up friends.
Sentiment Polarity: Support Gun non-Polarized

Sentence: not every act of violence needs to be about gun law , i see two idiots here .
Sentiment Polarity: Not relevant

Sentence: the second amendment is not absolute , it is subject to reasonable regulation .
Sentiment Polarity: Support Gun non-Polarized

Sentence: the right to bear arms is a fundamental human right , it is not subject to regulation .
Sentiment Polarized: Anti Gun Polarized

Sentence: the government should not be allowed to take away our ri

In [48]:
import re
from tqdm import tqdm

def annotate(dataset, user_prompt, examples):
    df_data = dataset.to_pandas()
    for i, val in enumerate(tqdm(dataset)):
        sentence = val['body_cleaned']

        # Make input
        text = user_prompt + "\n" + examples + "\n" + "Sentence: " + sentence + "\n" + "Sentiment Polarity:"
        inputs = tokenizer(text, return_tensors="pt").to("cuda")

        # Generate
        outputs = base_model.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=150, pad_token_id=tokenizer.eos_token_id)
        # print(tokenizer.decode(outputs[0], skip_special_tokens=True))
        outputs_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        matches = re.findall(r"Sentiment Polarity: (.+)", outputs_text)
        selected_sentiment = matches[2].strip() if len(matches) >= 3 else None
        print(selected_sentiment)

        # Update dataset
        df_data.loc[i, 'User label'] = selected_sentiment
        
    return df_data


In [49]:
new_dataset = annotate(dataset, user_prompt, examples)

  1%|          | 1/150 [00:12<30:37, 12.33s/it]

Support Gun Polarized


  1%|▏         | 2/150 [00:18<20:57,  8.49s/it]

Anti Gun Polarized


  2%|▏         | 3/150 [00:31<25:58, 10.60s/it]

...

Anti Gun Polarized


 99%|█████████▉| 149/150 [29:02<00:09,  9.85s/it]

Anti Gun Polarized


100%|██████████| 150/150 [29:12<00:00, 11.69s/it]

Anti Gun Polarized


In [50]:
new_dataset

,id,body_cleaned,User label,author,subreddit,predicted_community,score,created_utc
0,f4acjs9,"as a federal leo , the very idea of confiscati...",Support Gun Polarized,MDeXY,progun,0,454,1571492410
1,f56wn75,we must ban this horrible weapon of war,Anti Gun Polarized,MAp6s,progun,0,437,1572028587
2,f9nx9zd,`` you 're pretty stupid to not just have arme...,Anti Gun Polarized,a9GR9,progun,0,343,1575492221
3,dwx41ns,"meanwhile in england , an old man is being cha...",Anti Gun non-Polarized,qhsBh,progun,0,339,1523030793
4,ekz86fj,thanks but no thanks .,Neutral,L7DdD,progun,0,311,1555369507
...,...,...,...,...,...,...,...,...
145,ew7u70o,`` i respect the flag . that 's why i always f...,Support Gun Polarized,I35V7,GunsAreCool,3,48,1565197239
146,cxku7v9,mental health guys we need to do something abo...,Not relevant,n5s7P,GunsAreCool,3,47,1449090536
147,dv8iym6,what ? a nazi waving a gun around ? prepostero...,Anti Gun Polarized,mYqmA,GunsAreCool,3,47,1520289868
148,cyptaqf,i like how they keep switching from mass murde...,Anti Gun Polarized,K5aJM,GunsAreCool,3,46,1452202132


In [51]:
new_dataset.to_json('../data/polito-llama2-13b-chat-few-inference.json', orient='records', lines=True)